### Initializing CMBAgent

In [118]:
import os
import re
import numpy as np
import pandas as pd
import copy
import os
import glob
import json

os.environ["CMBAGENT_DEBUG"] = "false"
os.environ["ASTROPILOT_DISABLE_DISPLAY"] = "false"

import cmbagent
path_to_targets = '/Users/milind24/cmbagentmain/cmbagent'

### Making and Testing the samples to run

In [220]:
# Install needed packages if not already installed
# !pip install datasets

from datasets import load_dataset
import traceback

# 1. Load top 10 samples from DS-1000
ds = load_dataset("xlangai/DS-1000", split="test")
#samples = ds.shuffle(seed=42).select(range(1))  # Top 10 random samples
#samples = ds.select(range(1))
#samples = ds.select([35])
samples = ds.select(list(range(30, 41)))

### The prompt which goes into CMBAgent along with other instructiions

In [223]:
print(f"{samples['prompt'][8]}")

Problem:
I have pandas df with say, 100 rows, 10 columns, (actual data is huge). I also have row_index list which contains, which rows to be considered to take sum. I want to calculate sum on say columns 2,5,6,7 and 8. Can we do it with some function for dataframe object?
What I know is do a for loop, get value of row for each element in row_index and keep doing sum. Do we have some direct function where we can pass row_list, and column_list and axis, for ex df.sumAdvance(row_list,column_list,axis=0) ?
I have seen DataFrame.sum() but it didn't help I guess.
  a b c d q 
0 1 2 3 0 5
1 1 2 3 4 5
2 1 1 1 6 1
3 1 0 0 0 0

I want sum of 0, 2, 3 rows for each a, b, d columns 
a    3.0
b    3.0
d    6.0

Then I want to delete the largest one. Desired:

a    3.0
b    3.0

A:
<code>
import pandas as pd

df = pd.DataFrame({'a':[1,1,1,1],'b':[2,2,1,0],'c':[3,3,1,0],'d':[0,4,6,0],'q':[5,5,1,0]})
row_list = [0,2,3]
column_list = ['a','b','d']
</code>
result = ... # put solution in this variable
BEG

### The answer which goes into the Context Code to run and check it works or not!

In [209]:
print(f"{samples['reference_code'][0]}")

def g(df):
    return df.groupby('group').agg(lambda x : x.head(1) if x.dtype=='object' else x.mean() if x.name.endswith('2') else x.sum())

result = g(df.copy())



In [210]:
print(f"{samples['metadata'][0]}")

{'problem_id': 35, 'library_problem_id': 35, 'library': 'Pandas', 'test_case_cnt': 2, 'perturbation_type': 'Difficult-Rewrite', 'perturbation_origin_id': 33}


### The code context is the code to which the solution code gets attached and run to check our solution works or not

In [211]:
print(f"{samples['code_context'][0]}")

import pandas as pd
import numpy as np
import copy


def generate_test_case(test_case_id):
    def generate_ans(data):
        df = data
        return df.groupby("group").agg(
            lambda x: (
                x.head(1)
                if x.dtype == "object"
                else x.mean() if x.name.endswith("2") else x.sum()
            )
        )

    def define_test_input(test_case_id):
        if test_case_id == 1:
            df = pd.DataFrame(
                {
                    "group": ["A", "A", "A", "B", "B"],
                    "group_color": ["green", "green", "green", "blue", "blue"],
                    "val1": [5, 2, 3, 4, 5],
                    "val2": [4, 2, 8, 5, 7],
                    "val42": [1, 1, 4, 5, 1],
                }
            )
        if test_case_id == 2:
            df = pd.DataFrame(
                {
                    "group": ["A", "A", "A", "B", "B"],
                    "group_color": ["green", "green", "green", "blue", "blue"],
   

In [212]:
print(f"{samples['prompt'][0]}")

Problem:
I have a script that generates a pandas data frame with a varying number of value columns. As an example, this df might be
import pandas as pd
df = pd.DataFrame({
'group': ['A', 'A', 'A', 'B', 'B'],
'group_color' : ['green', 'green', 'green', 'blue', 'blue'],
'val1': [5, 2, 3, 4, 5], 
'val2' : [4, 2, 8, 5, 7]
})
  group group_color  val1  val2   val32
0     A       green     5     4     4
1     A       green     2     2     2
2     A       green     3     8     8
3     B        blue     4     5     5
4     B        blue     5     7     7


My goal is to get the grouped mean for each of the value columns which end with '2' and get the grouped sum for others.
df.groupby('group').agg({"group_color": "first", "val1": "sum", "val2": "mean", "val32": "mean"})

      group_color      val1      val2    val32
group                                
A           green  10.0  4.666667   4.666667
B            blue  9.0  6.000000   6.000000


but that does not work when the data frame in ques

## Helper Functions to execute our tasks

In [224]:
def extract_from_tags(text: str, start_tag: str, end_tag: str) -> str:
    start_index = len(start_tag) if text.startswith(start_tag) else 0
    end_index = text.find(end_tag, len(start_tag))
    end_index = end_index if end_index != -1 else len(text)
    return text[start_index:end_index]

def postprocess(code: str) -> str:
    
    code = extract_from_tags(code, "```python\n", "\n```")
    code = extract_from_tags(code, "```\n", "\n```")  # new pattern
    code = extract_from_tags(code, "<code>", "</code>")
    code = extract_from_tags(code, "", "</code>")
    code = extract_from_tags(code, "", "\nEND SOLUTION")
    code = extract_from_tags(code, "", "\n### END SOLUTION")
    return code.strip()

def extract_between_closing_and_opening(text: str, closing_tag: str, opening_tag: str) -> str:
    start_index = text.find(closing_tag)
    if start_index == -1:
        return ""
    start_index += len(closing_tag)
    end_index = text.find(opening_tag, start_index)
    if end_index == -1:
        return text[start_index:].strip()
    return text[start_index:end_index].strip()



In [225]:
def get_solution(cmbagent_results):
    chat_history = cmbagent_results['chat_history']
    final_context = cmbagent_results['final_context']

    # Load target file path from metadata and read target answer
    #target_file = metadata['target_file_path']
    # target_file = "targets/problem_{index}.txt"
    # try:
    #     with open(target_file, 'r') as f:
    #         target_answer = f.read().strip()
    # except FileNotFoundError as e:
    #     return ["FAILED", f"Target file not found: {e}"]

    # Build path to result.txt
    result_path = os.path.join(
        final_context['work_dir'],
        #final_context['database_path'],
        "result.txt"
    )
    result_path = os.path.abspath(result_path)

    # Read result answer
    try:
        with open(result_path, 'r') as f:
            result_answer = f.read().strip()
            return result_answer
    except FileNotFoundError as e:
        return f"Result file not found: {e}"

In [226]:
def my_agent(task, metadata):

    results = cmbagent.one_shot(task,
                                max_rounds=10,
                                agent='researcher',
                                #initial_agent=metadata['initial_agent'],
                                researcher_model='gpt-4o-mini',
                                work_dir="/Users/milind24/cmbagentmain/cmbagent/output/data/"
                                )


    return get_solution(results)
    #return results

In [227]:
results = []

for i, sample in enumerate(samples):
    print(f"### Sample {i+1}: {sample['metadata']['problem_id']} ###")
    # print(f"Write a short code following the given format and indentation. "
    # f"Place the executable code between <code> and </code> tags, without any other non-executable things \n"
    # f"Also save the code you place between <code> and </code> tags in a result.txt file "
    # f"Only provide the code completion needed. Don't repeat the context code or add any unnecessary lines or comments."
    # f"Prompt:\n {sample['prompt']}")

    # Run agent
    raw_output = my_agent( f"Write a short code following the given format and indentation. "
    f"Place the executable code between <code> and </code> tags, without any other non-executable things \n"
    f"Also save the code you place between <code> and </code> tags in a result.txt file \n"
    f"Only provide the code completion needed. Don't repeat the context code or add any unnecessary lines or comments.. \n"
    f"Prompt:\n {sample['prompt']}\n"
    , sample["metadata"]
    )
    #print(raw_output)  # Check structur
    
    #solution_code = postprocess(raw_output)
    solution_code = extract_between_closing_and_opening(raw_output,"<code>", "</code>")
    
    print("\nAgent Solution:\n", solution_code)

    # Build the full test script
    code_context = sample["code_context"]
    full_code = (
        f"{code_context}\n"
        + f"solution = '''{solution_code}'''\n"
        + "test_execution(solution)\n"
    )
    if "test_string(" in code_context:
        full_code += "test_string(solution)\n"

    print("\n--- Running Test ---")
    try:
        exec_locals = {}
        #exec(full_code, {}, exec_locals)
        exec(full_code)
        print("✅ Passed")
        results.append(("C", sample["metadata"]["problem_id"]))
    except Exception as e:
        print("❌ Failed")
        traceback.print_exc()
        results.append(("I", sample["metadata"]["problem_id"]))
    
    print("="*80)


### Sample 1: 30 ###


/opt/anaconda3/envs/cmbagentmain2/lib/python3.12/site-packages/autogen/agentchat/conversable_agent.py:129: UserWarning: Update function string contains no variables. This is probably unintended.
  warnings.warn("Update function string contains no variables. This is probably unintended.")


_User (to chat_manager):

Write a short code following the given format and indentation. Place the executable code between <code> and </code> tags, without any other non-executable things 
Also save the code you place between <code> and </code> tags in a result.txt file 
Only provide the code completion needed. Don't repeat the context code or add any unnecessary lines or comments.. 
Prompt:
 Problem:
Considering a simple df:
HeaderA | HeaderB | HeaderC 
    476      4365      457


Is there a way to rename all columns, for example to add to all columns an "X" in the end? 
HeaderAX | HeaderBX | HeaderCX 
    476      4365      457


I am concatenating multiple dataframes and want to easily differentiate the columns dependent on which dataset they came from. 
Or is this the only way?
df.rename(columns={'HeaderA': 'HeaderAX'}, inplace=True)


I have over 50 column headers and ten files; so the above approach will take a long time. 
Thank You


A:
<code>
import pandas as pd


df = pd.Data

Model,agent,Cost,Prompt Tokens,Completion Tokens,Total Tokens
gpt-4o-mini-2024-07-18,researcher,0.00013,777,20,797


researcher (to chat_manager):

<code>
df.rename(columns=lambda x: x + 'X', inplace=True)
</code>

--------------------------------------------------------------------------------

Next speaker: researcher_response_formatter


>>>>>>>> USING AUTO REPLY...


...

Model,agent,Cost,Prompt Tokens,Completion Tokens,Total Tokens
gpt-4o-2024-11-20,control,0.00327,1151,39,1190


control (to chat_manager):

***** Suggested tool call (call_075LGzuc3rGRJTc6nla9EHQs): record_status *****
Arguments: 
{"current_status":"failed","current_plan_step_number":1,"current_sub_task":"solve the main task.","current_instructions":"solve the main task.","agent_for_sub_task":"researcher"}
******************************************************************************

--------------------------------------------------------------------------------

Next speaker: _Group_Tool_Executor


>>>>>>>> EXECUTING FUNCTION record_status...
Call ID: call_075LGzuc3rGRJTc6nla9EHQs
Input arguments: {'current_status': 'failed', 'current_plan_step_number': 1, 'current_sub_task': 'solve the main task.', 'current_instructions': 'solve the main task.', 'agent_for_sub_task': 'researcher'}
_Group_Tool_Executor (to chat_manager):

***** Response from calling tool (call_075LGzuc3rGRJTc6nla9EHQs) *****

**Step number:** 1 out of 1.
 
**Sub-task:** solve the main task.
 
**Agent in charge of sub-task:** 

Model,agent,Cost,Prompt Tokens,Completion Tokens,Total Tokens
o3-mini-2025-01-31,researcher_response_formatter,0.00473,1312,748,2060


researcher_response_formatter (to chat_manager):

**Markdown:**

```markdown
<!-- filename: result.txt -->
<code>
result = df.isnull().sum().replace(0, float('nan'))
</code>
```

--------------------------------------------------------------------------------

Next speaker: executor


>>>>>>>> EXECUTING CODE BLOCK (inferred language is markdown)...
executor (to chat_manager):

execution results:
Execution output: Content saved to /Users/milind24/cmbagentmain/cmbagent/output/data/result.txt


--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (b0a3ad32-396a-4a98-8b31-1eeec9c374d6): Maximum rounds (10) reached

Displaying cost…

| Agent                         | Cost ($)    | Prompt Tokens | Completion Tokens | Total Tokens |
|:------------------------------|------------:|--------------:|------------------:|-------------:|
| executor response formatter   | $0.00171750 |           631 |                14 |          645 |
| researcher

Traceback (most recent call last):
  File "/var/folders/f6/v55nk5k10bl5v1zpkfhq51h00000gn/T/ipykernel_46758/2004163890.py", line 40, in <module>
    exec(full_code)
  File "<string>", line 56, in <module>
  File "<string>", line 50, in test_execution
  File "<string>", line 27, in generate_test_case
  File "<string>", line 9, in generate_ans
  File "/opt/anaconda3/envs/cmbagentmain2/lib/python3.12/site-packages/pandas/core/frame.py", line 10374, in apply
    return op.apply().__finalize__(self, method="apply")
           ^^^^^^^^^^
  File "/opt/anaconda3/envs/cmbagentmain2/lib/python3.12/site-packages/pandas/core/apply.py", line 916, in apply
    return self.apply_standard()
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/cmbagentmain2/lib/python3.12/site-packages/pandas/core/apply.py", line 1068, in apply_standard
    return self.wrap_results(results, res_index)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/cmbagentmain2/lib/python3.12/site-

# Results --> Correct : C :: Incorrect : I

In [228]:
results

[('C', 30),
 ('C', 31),
 ('I', 32),
 ('C', 33),
 ('C', 34),
 ('C', 35),
 ('C', 36),
 ('C', 37),
 ('C', 38),
 ('I', 39),
 ('I', 40)]

In [230]:
RED = '\033[91m'
RESET = '\033[0m'

print(f"{'Problem Number':<15} {'Result':<10}")
print("-" * 26)
for status, number in results:
    if status == 'C':
        result_str = "Passed"
    else:
        result_str = f"{RED}Failed{RESET}"
    print(f"{number:<15} {result_str:<10}")

Problem Number  Result    
--------------------------
30              Passed    
31              Passed    
32              Failed
33              Passed    
34              Passed    
35              Passed    
36              Passed    
37              Passed    
38              Passed    
39              Failed
40              Failed


### Putting the original solution into my code logic to check my code and logic works or not and it passes gracefully

In [150]:
for i, sample in enumerate(samples):
    # print(f"### Sample {i+1}: {sample['metadata']['problem_id']} ###")
    # print(f"Write a short code following the given format and indentation. "
    # f"Place the executable code between <code> and </code> tags, without any other non-executable things \n"
    # f"Also save the code you place between <code> and </code> tags along with the tags in a result.txt file "
    # f"Prompt:\n {sample['prompt']}")

    # # Run agent
    # raw_output = my_agent( f"Write a short code following the given format and indentation. "
    # f"Place the executable code between <code> and </code> tags, without any other non-executable things \n"
    # f"Also save the code you place between <code> and </code> tags along with the tags in a result.txt file \n"
    # f"Prompt:\n {sample['prompt']}\n"
    # f"Finally save the code you place between <code> and </code> tags along with the tags in a \"result.txt\" file "
    # , sample["metadata"]
    # )
    # #print(raw_output)  # Check structur
    
    # solution_code = postprocess(raw_output)
    #solution_code = raw_output
    solution_code = '''def g(df, List):
     return df.iloc[List]
result = g(df.copy(), List)'''



    print("\nAgent Solution:\n", solution_code)

    # Build the full test script
    code_context = sample["code_context"]
    full_code = (
        f"{code_context}\n"
        + f"solution = '''{solution_code}'''\n"
        + "test_execution(solution)\n"
    )
    if "test_string(" in code_context:
        full_code += "test_string(solution)\n"

    print("\n--- Running Test ---")
    try:
        exec_locals = {}
        #exec(full_code, {}, exec_locals)
        exec(full_code)
        print("✅ Passed")
        results.append(("C", sample["metadata"]["problem_id"]))
    except Exception as e:
        print("❌ Failed")
        traceback.print_exc()
        results.append(("I", sample["metadata"]["problem_id"]))
    
    print("="*80)


Agent Solution:
 def g(df, List):
     return df.iloc[List]
result = g(df.copy(), List)

--- Running Test ---
✅ Passed


In [151]:
print(f"{full_code}")

import pandas as pd
import numpy as np
import copy


def generate_test_case(test_case_id):
    def generate_ans(data):
        data = data
        df, List = data
        return df.iloc[List]

    def define_test_input(test_case_id):
        if test_case_id == 1:
            df = pd.DataFrame(
                {
                    "Col1": [1, 4, 7, 10, 13, 16],
                    "Col2": [2, 5, 8, 11, 14, 17],
                    "Col3": [3, 6, 9, 12, 15, 18],
                    "Type": [1, 1, 2, 2, 3, 3],
                }
            )
            List = np.random.permutation(len(df))
        return df, List

    test_input = define_test_input(test_case_id)
    expected_result = generate_ans(copy.deepcopy(test_input))
    return test_input, expected_result


def exec_test(result, ans):
    try:
        pd.testing.assert_frame_equal(result, ans, check_dtype=False)
        return 1
    except:
        return 0


exec_context = r"""
import pandas as pd
import numpy as np
df, List = te

In [152]:
exec(full_code)
print("✅ Passed")

✅ Passed


In [145]:
'''def g(df, List):
     return df.iloc[List]
result = g(df.copy(), List)'''

'def g(df, List):\n     return df.iloc[List]\nresult = g(df.copy(), List)'